# 셀레니움으로 금감원 비디오자료 자동 다운로드 구현

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import requests, os

DOWNLOAD_DIR = "videos"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)


# 드라이버 초기화 함수
def init_driver():
    service = Service(ChromeDriverManager().install())
    opts = webdriver.ChromeOptions()
    opts.add_argument("--headless")
    return webdriver.Chrome(service=service, options=opts)


driver = init_driver()

for page in range(1, 13):
    try:
        video_page_urls = []
        url = f"https://www.fss.or.kr/fss/bbs/B0000203/list.do?menuNo=200686&pageIndex={page}"
        driver.get(url)

        # 링크 수집
        elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "div.bd-list-thumb-a a")
            )
        )
        for el in elements:
            video_page_urls.append(el.get_attribute("href"))

        for idx, post_url in enumerate(video_page_urls, 1):
            driver.get(post_url)
            title = (
                WebDriverWait(driver, 10)
                .until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.name")))
                .text
            )
            video = driver.find_element(By.TAG_NAME, "video").get_attribute("src")
            filename = "".join(
                c for c in title if c.isalnum() or c in (" ", "_")
            ).replace(" ", "_")
            path = os.path.join(DOWNLOAD_DIR, f"{filename}.mp4")

            r = requests.get(video, stream=True, timeout=30)
            if r.status_code == 200:
                with open(path, "wb") as f:
                    for chunk in r.iter_content(8192):
                        f.write(chunk)
    except WebDriverException:
        driver.quit()
        driver = init_driver()
    except Exception as e:
        print(f"오류: {e}")

driver.quit()